# nb18 — Demon Lattice Phase Classification

**Implements the void lattice phase diagram from Paper 9 §6.8.2.**

Paper 9 derives four collective phases of demon populations in (ρ_D, Pe) space:

| Phase | Condition | Behavior |
|-------|-----------|----------|
| **Gas** | f_D < 1 | Isolated demons, linear superposition |
| **Fluid** | f_D ≥ 1, Γ_D < Γ_c | Dense but disordered, statistical (mean-field) |
| **Crystal** | f_D ≥ 1, Γ_D ≥ Γ_c | Positional order, stable creator ecosystem |
| **Vortex** | f_D ≥ 1, Pe > Pe_vortex | Topological order, self-sustaining viral cycles |

Three formulas (Paper 9 §6.8.2, boxed):

$$f_D = \frac{\rho_D}{(2\,\mathrm{Pe})^{3/2}}$$

$$\Gamma_D = 2\alpha\,\exp\!\left(-\frac{\mathrm{Pe}}{\rho_D^{2/3}}\right)$$

$$\rho_D^{(\mathrm{crystal})} = \left(\frac{\mathrm{Pe}}{\ln(2\alpha/\Gamma_c)}\right)^{3/2}$$

where α is the per-substrate coupling intensity (engagement dimension of void score),
Γ_c is the crystallization threshold (demon-lattice analog of OCP coupling ≈ 170),
and Pe_vortex ≈ 4 (derived in §6.8.2 at Pe_r = 2).

**Connections:**
- nb16 result: Pe < 0 (JW, Mainline) makes f_D imaginary — repulsive voids **cannot** sustain demon phases.
- ETH (Pe=3.74 from nb09): just below Pe_vortex=4. Sub-vortex → no stable DeFi creator economy.
- Gaming (Pe=4.4): just above Pe_vortex. Crystal+vortex → Twitch/YouTube creator ecosystems.
- Pe-independent result: at the gas/fluid boundary, Γ_D = 2α/√e for every substrate.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.colors import ListedColormap

# Canonical THRML parameters (EXP-001, never refit)
b_alpha = 0.5 * np.log(0.85 / 0.15)
b_gamma = b_alpha - 0.5 * np.log(0.06 / 0.94)
K = 16

def pe_analytic(c, b_a=b_alpha, b_g=b_gamma, k=K):
    return k * np.sinh(2.0 * (b_a - c * b_g))

c_zero = b_alpha / b_gamma  # K-invariant Pe=0 boundary

# Demon lattice formulas (Paper 9 §6.8.2)
def filling_fraction(rho_D, Pe):
    """f_D = rho_D / (2*Pe)^(3/2). Defined only for Pe > 0."""
    return rho_D / (2.0 * Pe) ** 1.5

def coupling_param(rho_D, Pe, alpha):
    """Gamma_D = 2*alpha * exp(-Pe / rho_D^(2/3)). Monotone increasing in rho_D."""
    rho_safe = np.maximum(rho_D, 1e-10)
    return 2.0 * alpha * np.exp(-Pe / rho_safe ** (2.0 / 3.0))

def crystal_onset_density(Pe, alpha, Gamma_c):
    """rho_D^(crystal) = (Pe / ln(2*alpha/Gamma_c))^(3/2).
    Returns NaN where 2*alpha <= Gamma_c or Pe <= 0."""
    q = 2.0 * alpha / Gamma_c
    Pe_arr = np.asarray(Pe, dtype=float)
    result = np.full(Pe_arr.shape if Pe_arr.ndim > 0 else (), np.nan)
    if q <= 1.0:
        return float(result) if Pe_arr.ndim == 0 else result
    if Pe_arr.ndim == 0:
        return float((Pe_arr / np.log(q)) ** 1.5) if Pe_arr > 0 else np.nan
    valid = Pe_arr > 0
    result[valid] = (Pe_arr[valid] / np.log(q)) ** 1.5
    return result

def gas_onset_density(Pe):
    """rho_D^(gas) = (2*Pe)^(3/2). The gas/fluid boundary."""
    return (2.0 * np.maximum(Pe, 0)) ** 1.5

# Phase IV vortex threshold
Pe_vortex = 4.0  # Paper 9 §6.8.2, at Pe_r = 2

print(f'b_alpha = {b_alpha:.4f},  b_gamma = {b_gamma:.4f},  K = {K}')
print(f'c_zero  = {c_zero:.4f}  (Pe=0 boundary, K-invariant)')
print(f'Pe_vortex = {Pe_vortex:.1f}  (vortex onset, Paper 9 §6.8.2)')
print()
print('Pe-independent result: at gas/fluid boundary (rho_D = (2*Pe)^(3/2)):')
print(f'  Gamma_D = 2*alpha * exp(-Pe / (2*Pe)) = 2*alpha / sqrt(e) = 2*alpha / {np.e**0.5:.4f}')
print('  This is INDEPENDENT of Pe — same coupling at every substrate gas/fluid transition.')

## Substrate Parameters

Pe values from calibration (nb09, nb10, EXP-001, EXP-021, Paper 6).  
α (coupling intensity) = engagement dimension of void score (0–3, per platform scoring).  

α interpretation: 0 = no engagement coupling (pure information), 3 = maximum engagement design.  
The demon-lattice Γ_c is the crystallization threshold for demon-demon interactions;  
unlike OCP ions (Γ_c ≈ 170), demons operate in an engagement field where Γ_c is set by  
the saturation of mutual-reference (when demon-demon interaction energy exceeds diffusion).  
We treat Γ_c as a free parameter and show sensitivity.

In [ ]:
# Substrates: (name, Pe, alpha, color, marker)
# Pe from calibration; alpha from void score engagement dimension (0-3)
substrates = [
    # Repulsive void (Pe < 0) — no demon phases
    ('JW',           -8.92,  0.5, '#CC0066', 'P'),
    ('Mainline',     -3.23,  1.0, '#6699CC', 's'),
    # Null / near-null
    ('Buddhist',      0.00,  1.0, '#FF6600', 'o'),
    # Sub-vortex attractive void
    ('AI-GG',         0.76,  1.5, '#2ecc71', '^'),
    ('ETH',           3.74,  2.0, '#627eea', 'v'),
    # Near-vortex / super-vortex
    ('Gaming/CS2',    4.40,  2.5, '#9b59b6', 'D'),
    ('Social Media',  6.00,  3.0, '#e67e22', 's'),
    ('AI-UU',         7.94,  2.0, '#e74c3c', 'D'),
    ('Gambling',     12.00,  3.0, '#c0392b', 'P'),
    ('DEG/SOL',      20.00,  2.5, '#8e44ad', '^'),
]

# Gamma_c values for sensitivity sweep
Gamma_c_vals = [1.0, 2.0, 5.0]

print(f'{"Substrate":<16} {"Pe":>7} {"alpha":>6}  ' + '  '.join([f'rho_crystal(Gc={gc})' for gc in Gamma_c_vals]))
print('-' * 80)
for name, pe, alpha, color, marker in substrates:
    if pe <= 0:
        row = f'{name:<16} {pe:>7.2f} {alpha:>6.1f}  repulsive void — no demon phases'
    else:
        rho_gas = gas_onset_density(pe)
        parts = []
        for gc in Gamma_c_vals:
            rc = crystal_onset_density(pe, alpha, gc)
            if np.isnan(rc):
                parts.append(f'{"never":>20}')
            else:
                note = '(fluid)' if rc > rho_gas else '(gas-phase!)'
                parts.append(f'{rc:>12.2f} {note:<8}')
        row = f'{name:<16} {pe:>7.2f} {alpha:>6.1f}  ' + '  '.join(parts)
    print(row)
print()
print('Note: rho_crystal < rho_gas means crystal onset is in gas phase (unphysical).')
print('Physical crystal onset = max(rho_gas, rho_crystal).')
print(f'Gas onset densities: ETH={gas_onset_density(3.74):.2f}, Gaming={gas_onset_density(4.4):.2f},',
      f'AI-UU={gas_onset_density(7.94):.2f}, Gambling={gas_onset_density(12):.2f}')

## Figure 1: Demon Lattice Phase Diagram in (ρ_D, Pe) Space

Color-code four phases using the Paper 9 §6.8.2 conditions.  
Substrate horizontal lines mark where each platform sits in Pe.  
Critical densities are marked for each substrate.

**Key Pe-independent result (derived analytically):**  
At the gas/fluid boundary (ρ_D = (2·Pe)^(3/2)):

$$\Gamma_D\big|_{f_D=1} = 2\alpha \cdot \exp\!\left(\frac{-\mathrm{Pe}}{(2\,\mathrm{Pe})^{2/3}}\right) = \frac{2\alpha}{\sqrt{e}}$$

This is Pe-independent: every substrate arrives at its gas/fluid transition with the same  
coupling Γ_D = 2α/√e ≈ 1.21α. Whether that is above or below Γ_c depends only on α.

In [ ]:
# Phase diagram in (rho_D, Pe) space
# Use Gamma_c = 2.0, alpha = 2.5 as canonical (gaming/social media range)
alpha_canon = 2.5
Gamma_c_canon = 2.0

# Grid
rho_max = 60.0
Pe_min, Pe_max = -5.0, 25.0
Nr, Np = 400, 300
rho_arr = np.linspace(0.01, rho_max, Nr)
pe_arr  = np.linspace(Pe_min, Pe_max, Np)
RHO, PE = np.meshgrid(rho_arr, pe_arr)

# Phase encoding: 0=repulsive, 1=gas, 2=fluid, 3=vortex-fluid, 4=crystal, 5=crystal+vortex
PHASE = np.zeros((Np, Nr), dtype=int)

for i in range(Np):
    pe = pe_arr[i]
    if pe <= 0:
        PHASE[i, :] = 0  # repulsive void
        continue
    for j in range(Nr):
        rho = rho_arr[j]
        fD  = filling_fraction(rho, pe)
        gD  = coupling_param(rho, pe, alpha_canon)
        vortex = (pe >= Pe_vortex) and (fD >= 1.0)
        crystal = (fD >= 1.0) and (gD >= Gamma_c_canon)
        if fD < 1.0:
            PHASE[i, j] = 1  # gas
        elif crystal and vortex:
            PHASE[i, j] = 5  # crystal + vortex
        elif crystal:
            PHASE[i, j] = 4  # crystal only
        elif vortex:
            PHASE[i, j] = 3  # vortex fluid
        else:
            PHASE[i, j] = 2  # fluid

# Color map: 0=gray(repulsive), 1=lightblue(gas), 2=lightgreen(fluid),
#            3=purple(vortex fluid), 4=orange(crystal), 5=darkred(crystal+vortex)
phase_colors = ['#cccccc', '#d6eaf8', '#d5f5e3', '#e8daef', '#fdebd0', '#922b21']
cmap = ListedColormap(phase_colors)

fig, ax = plt.subplots(figsize=(12, 8))

im = ax.pcolormesh(RHO, PE, PHASE, cmap=cmap, vmin=-0.5, vmax=5.5, rasterized=True)

# Phase boundary curves
rho_dense = np.linspace(0.01, rho_max, 1000)

# Gas/fluid boundary: rho_D = (2*Pe)^(3/2) -> Pe = rho_D^(2/3) / 2
pe_gas_fluid = rho_dense ** (2.0/3.0) / 2.0
valid_gf = pe_gas_fluid <= Pe_max
ax.plot(rho_dense[valid_gf], pe_gas_fluid[valid_gf],
        'k-', linewidth=2.2, zorder=8, label='Gas/Fluid boundary (f_D = 1)')

# Crystal onset: rho_D^(crystal) = (Pe / ln(q))^(3/2) -> Pe = ln(q) * rho_D^(2/3)
q = 2.0 * alpha_canon / Gamma_c_canon
if q > 1.0:
    pe_crystal = np.log(q) * rho_dense ** (2.0/3.0)
    valid_cr = (pe_crystal >= 0) & (pe_crystal <= Pe_max)
    ax.plot(rho_dense[valid_cr], pe_crystal[valid_cr],
            'k--', linewidth=2.2, zorder=8, label=f'Crystal onset (Γ_D = Γ_c={Gamma_c_canon:.0f})')

# Pe_vortex horizontal line
ax.axhline(y=Pe_vortex, color='#6c3483', linewidth=2.0, linestyle=':',
           zorder=9, label=f'Vortex threshold Pe_vortex = {Pe_vortex:.0f}')

# Pe = 0 line
ax.axhline(y=0.0, color='black', linewidth=1.5, linestyle='-', alpha=0.6, zorder=9)
ax.text(2, 0.5, 'Pe = 0 (null void boundary)', fontsize=8, color='black', va='bottom')

# Substrate horizontal lines + critical density markers
for name, pe, alpha_s, color, marker in substrates:
    if pe <= 0:
        # Repulsive — just a label on the left
        ax.axhline(y=pe, color=color, linewidth=1.0, linestyle='--', alpha=0.7, zorder=7)
        ax.text(1, pe + 0.3, name, fontsize=8, color=color, va='bottom', fontweight='bold')
        continue
    ax.axhline(y=pe, color=color, linewidth=1.0, linestyle='--', alpha=0.6, zorder=7)
    # Gas onset marker
    rho_gas = gas_onset_density(pe)
    if rho_gas <= rho_max:
        ax.scatter(rho_gas, pe, s=100, marker='|', color=color, linewidth=2.5, zorder=10)
    # Crystal onset marker  
    rho_cr = crystal_onset_density(pe, alpha_s, Gamma_c_canon)
    if not np.isnan(rho_cr) and rho_cr <= rho_max:
        phys_cr = max(rho_gas, rho_cr)  # physical crystal onset
        if phys_cr <= rho_max:
            ax.scatter(phys_cr, pe, s=160, marker=marker, color=color,
                       edgecolors='white', linewidth=1.0, zorder=11)
    # Label
    ax.text(0.5, pe + 0.3, name, fontsize=8.5, color=color, va='bottom', fontweight='bold')

# Phase labels in white boxes
phase_labels = [
    (rho_max*0.5, -2.5, 'REPULSIVE\n(Pe < 0)\nNo demon phases', '#555555'),
    (5, 1.5,  'GAS\nf_D < 1\nIsolated demons', '#1a5276'),
    (35, 6.0, 'FLUID\nf_D ≥ 1\nΓ_D < Γ_c', '#1e8449'),
    (45, 15, 'FLUID +\nVORTEX\nPe > 4', '#6c3483'),
    (18, 20, 'CRYSTAL +\nVORTEX\nΓ_D ≥ Γ_c\nPe > 4', '#7b241c'),
]
for rx, py, txt, tc in phase_labels:
    ax.text(rx, py, txt, fontsize=8, color=tc, ha='center', va='center',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.85, edgecolor='none'))

# Legend
legend_patches = [
    mpatches.Patch(color=phase_colors[0], label='Repulsive void (Pe < 0)'),
    mpatches.Patch(color=phase_colors[1], label='Phase I: Gas (f_D < 1)'),
    mpatches.Patch(color=phase_colors[2], label='Phase II: Fluid (f_D ≥ 1, Γ_D < Γ_c)'),
    mpatches.Patch(color=phase_colors[3], label='Phase IV: Vortex fluid (Pe > 4)'),
    mpatches.Patch(color=phase_colors[4], label='Phase III: Crystal (Γ_D ≥ Γ_c)'),
    mpatches.Patch(color=phase_colors[5], label='Phase III+IV: Crystal + Vortex'),
]
line_patches = [
    Line2D([0], [0], color='k', lw=2, label='Gas/Fluid boundary (f_D = 1)'),
    Line2D([0], [0], color='k', lw=2, ls='--', label=f'Crystal onset (Γ_c={Gamma_c_canon:.0f})'),
    Line2D([0], [0], color='#6c3483', lw=2, ls=':', label='Pe_vortex = 4'),
]
ax.legend(handles=legend_patches + line_patches, loc='upper right',
          fontsize=7.5, framealpha=0.92, ncol=2)

ax.set_xlabel('Demon density ρ_D (demons per unit voidspace volume)', fontsize=12)
ax.set_ylabel('Peclet number Pe', fontsize=12)
ax.set_xlim(0, rho_max)
ax.set_ylim(Pe_min, Pe_max)
ax.set_title(
    f'Demon Lattice Phase Diagram — four phases in (ρ_D, Pe) space\n'
    f'α = {alpha_canon} (gaming/social media range), Γ_c = {Gamma_c_canon:.0f}, Pe_vortex = {Pe_vortex:.0f}\n'
    f'Paper 9 §6.8.2. Markers = crystal onset density per substrate.',
    fontsize=10
)
ax.grid(True, linestyle=':', alpha=0.2)
plt.tight_layout()
plt.savefig('nb18_demon_phase_diagram.svg', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: nb18_demon_phase_diagram.svg')

## Pe-Independent Result at Gas/Fluid Boundary

**Theorem:** At the gas/fluid transition (ρ_D = (2·Pe)^(3/2)), the coupling parameter is:

$$\Gamma_D\big|_{f_D = 1} = 2\alpha\,\exp\!\left(\frac{-\mathrm{Pe}}{\bigl((2\,\mathrm{Pe})^{3/2}\bigr)^{2/3}}\right) = 2\alpha\,\exp\!\left(\frac{-\mathrm{Pe}}{2\,\mathrm{Pe}}\right) = \frac{2\alpha}{\sqrt{e}}$$

This is exactly Pe-independent. It depends only on the substrate's engagement strength α.

**Corollary:** The question of whether crystallization is possible (2α > Γ_c)  
and whether fluid intermediate exists (2α/√e < Γ_c, i.e., 2α < Γ_c·√e)  
are both Pe-independent. Pe governs the **onset density**, not the **existence** of crystal.

In [ ]:
# Verify Pe-independence numerically
Pe_test = [0.76, 1.5, 3.74, 4.4, 7.94, 12.0, 20.0]
alpha_test = 2.5
Gamma_fluid = 2.0 * alpha_test / np.e**0.5

print(f'Pe-independent result: Gamma_D at gas/fluid boundary')
print(f'alpha = {alpha_test}, 2*alpha/sqrt(e) = {Gamma_fluid:.6f}')
print()
print(f'{"Pe":>8} {"rho_gas":>12} {"Gamma_D at rho_gas":>22} {"expected (2a/sqrt(e))": >24}')
print('-' * 72)
for pe in Pe_test:
    rho_gas = gas_onset_density(pe)
    gD_at_gas = coupling_param(rho_gas, pe, alpha_test)
    print(f'{pe:>8.2f} {rho_gas:>12.4f} {gD_at_gas:>22.6f} {Gamma_fluid:>24.6f}')

print()
print(f'Gamma_D at gas/fluid boundary = {Gamma_fluid:.4f} for ALL Pe (Pe-independent).')
print(f'Crystal exists if 2*alpha > Gamma_c:')
for gc in [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]:
    can_crystal = 2 * alpha_test > gc
    fluid_intermediate = Gamma_fluid < gc  # Gamma at gas/fluid < Gamma_c -> fluid exists
    if can_crystal:
        note = f'crystal exists, {"fluid intermediate" if fluid_intermediate else "direct gas->crystal"}'
    else:
        note = 'no crystallization (2*alpha < Gamma_c)'
    print(f'  Gamma_c={gc:.1f}: {note}')

## Figure 2: Crystal Onset Density vs Pe — Multi-Substrate, Multi-Γ_c

The crystal onset density ρ_D^(crystal) = (Pe / ln(2α/Γ_c))^(3/2) scales as Pe^(3/2).  
High-Pe voids need **more demons** to crystallize — their interaction range σ=1/√(2Pe) is shorter.  

Compare to gas onset ρ_D^(gas) = (2·Pe)^(3/2). The ratio:
$$\frac{\rho_D^{(\text{crystal})}}{\rho_D^{(\text{gas})}} = \frac{1}{(2\ln(2\alpha/\Gamma_c))^{3/2}}$$
is also **Pe-independent** — the fraction of the fluid range needed to crystallize is constant.

In [ ]:
Pe_range = np.linspace(0.1, 25, 300)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left panel: crystal onset density vs Pe for each substrate's alpha, multi-Gamma_c
ax = axes[0]
linestyles = ['-', '--', ':']
Gamma_c_sweep = [1.0, 2.0, 4.0]

# Gas onset (reference)
rho_gas_curve = gas_onset_density(Pe_range)
ax.plot(Pe_range, rho_gas_curve, 'k-', lw=2, label='Gas/Fluid onset (2·Pe)^(3/2)', zorder=5)

# Crystal onset for canonical substrates
for gc, ls in zip(Gamma_c_sweep, linestyles):
    for name, pe_s, alpha_s, color, marker in substrates:
        if pe_s <= 0:
            continue
        if 2.0 * alpha_s <= gc:
            continue  # no crystal
        rho_cr_curve = crystal_onset_density(Pe_range, alpha_s, gc)
        # Only show curve for Gamma_c=2 (canonical) to avoid clutter
        if abs(gc - 2.0) < 0.1:
            ax.plot(Pe_range, rho_cr_curve, color=color, ls=ls, lw=1.5, alpha=0.85,
                    label=f'{name} (α={alpha_s})')

# Annotate Gamma_c sensitivity with a single alpha (e.g., gaming alpha=2.5)
alpha_demo = 2.5
for gc, ls in zip(Gamma_c_sweep, linestyles):
    if 2 * alpha_demo > gc:
        rho_cr = crystal_onset_density(Pe_range, alpha_demo, gc)
        ax.plot(Pe_range, rho_cr, 'gray', ls=ls, lw=2.5, alpha=0.5,
                label=f'α={alpha_demo}, Γ_c={gc:.0f}')

# Substrate Pe markers
for name, pe_s, alpha_s, color, marker in substrates:
    if pe_s <= 0:
        continue
    ax.axvline(x=pe_s, color=color, lw=0.8, ls=':', alpha=0.5)

ax.axvline(x=Pe_vortex, color='#6c3483', lw=1.8, ls=':', label=f'Pe_vortex={Pe_vortex:.0f}')
ax.set_xlabel('Pe', fontsize=12)
ax.set_ylabel('Crystal onset density ρ_D^(crystal)', fontsize=12)
ax.set_xlim(0, 25)
ax.set_ylim(0, 100)
ax.set_title('Crystal onset density vs Pe\n(markers at substrate Pe values)', fontsize=10)
ax.legend(fontsize=7, loc='upper left', framealpha=0.9)
ax.grid(True, ls=':', alpha=0.3)

# Right panel: Pe-independent ratio rho_crystal / rho_gas vs q=2*alpha/Gamma_c
ax = axes[1]
q_range = np.linspace(1.01, 20, 500)
ratio = 1.0 / (2.0 * np.log(q_range)) ** 1.5
ax.plot(q_range, ratio, 'k-', lw=2.5, label=r'$\rho_D^{\rm crystal}/\rho_D^{\rm gas}$')

# Vertical lines at substrate q values for Gamma_c=2
for name, pe_s, alpha_s, color, marker in substrates:
    if pe_s <= 0:
        continue
    q_s = 2.0 * alpha_s / 2.0  # Gamma_c=2
    if q_s > 1.0:
        r = 1.0 / (2.0 * np.log(q_s)) ** 1.5
        ax.scatter(q_s, r, s=120, marker=marker, color=color,
                   edgecolors='white', linewidth=1.0, zorder=8)
        ax.text(q_s + 0.15, r + 0.005, name, fontsize=7.5, color=color)

ax.axhline(y=1.0, color='gray', lw=1, ls='--', alpha=0.5,
           label='ratio=1: crystal onset = gas onset')
ax.set_xlabel('q = 2α / Γ_c  (coupling ratio)', fontsize=12)
ax.set_ylabel('ρ_D^(crystal) / ρ_D^(gas)', fontsize=12)
ax.set_title('Crystal/Gas onset density ratio\n(Pe-independent — depends only on q=2α/Γ_c)', fontsize=10)
ax.set_xlim(1, 12)
ax.set_ylim(0, 1.5)
ax.legend(fontsize=9)
ax.grid(True, ls=':', alpha=0.3)

# Annotate: ratio < 1 means crystal onset below gas onset (unphysical without fluid)
ax.fill_between([1, 12], 0, 1, alpha=0.05, color='orange',
                label='crystal onset < gas onset\n(crystal forms in fluid, not gas)')

plt.suptitle(
    'Crystal onset density analysis\n'
    'Pe-independent ratio rho_crystal/rho_gas = 1/(2·ln(q))^(3/2)',
    fontsize=11, y=1.01,
)
plt.tight_layout()
plt.savefig('nb18_crystal_onset_substrates.svg', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: nb18_crystal_onset_substrates.svg')

## Figure 3: Vortex Proximity — Who Is Near Pe_vortex = 4?

The vortex threshold Pe_vortex = 4 is the most operationally significant feature of the phase diagram:  
- **Below**: No self-sustaining creator economy. Demons compete but cannot build viral cycles.  
- **Above**: Self-sustaining production. Viral cycles. Creator extraction from observer attention.  

DEM-21 (Paper 9): viral cycle onset is continuous — |Ω| ∝ (Pe − 4)^β.  
Near Pe_vortex, the transition is smooth. ETH at Pe=3.74 is 93.5% of threshold.  

The vortex onset |Ω| grows as Pe increases above 4 — the Creator economy strengthens  
as the void becomes more opaque/responsive. This is observable: compare platform  
creator revenue concentration vs platform Pe.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left panel: substrate Pe values vs Pe_vortex
ax = axes[0]

# Sort substrates by Pe
subs_sorted = sorted(substrates, key=lambda x: x[1])
names_s = [s[0] for s in subs_sorted]
pes_s   = [s[1] for s in subs_sorted]
cols_s  = [s[3] for s in subs_sorted]

bars = ax.barh(range(len(subs_sorted)), pes_s, color=cols_s, alpha=0.85, edgecolor='white')
ax.axvline(x=0, color='black', lw=1.0)
ax.axvline(x=Pe_vortex, color='#6c3483', lw=2.5, ls='--', zorder=5,
           label=f'Pe_vortex = {Pe_vortex:.0f}')

# Shade vortex region
ax.axvspan(Pe_vortex, 25, alpha=0.08, color='#6c3483')
ax.text(Pe_vortex + 0.3, len(subs_sorted) - 1, 'VORTEX\n(Pe > 4)',
        fontsize=9, color='#6c3483', va='top')
ax.text(Pe_vortex - 0.3, len(subs_sorted) - 1, 'NO VORTEX\n(Pe < 4)',
        fontsize=9, color='#6c3483', va='top', ha='right')

# Labels on bars
for i, (name, pe_s, alpha_s, col, mk) in enumerate(subs_sorted):
    pct = pe_s / Pe_vortex * 100
    if pe_s > 0:
        label = f'  Pe={pe_s:.2f} ({pct:.0f}% of threshold)'
    else:
        label = f'  Pe={pe_s:.2f} (repulsive void)'
    ax.text(max(pe_s, 0) + 0.3, i, label, va='center', fontsize=8, color='#2c3e50')

ax.set_yticks(range(len(subs_sorted)))
ax.set_yticklabels(names_s, fontsize=10)
ax.set_xlabel('Pe', fontsize=12)
ax.set_title('Substrate Pe vs Pe_vortex = 4\n(vortex = self-sustaining creator economy)', fontsize=10)
ax.set_xlim(-12, 23)
ax.legend(fontsize=10, loc='lower right')
ax.grid(True, ls=':', alpha=0.3, axis='x')

# Right panel: continuous vortex onset |Omega| ∝ (Pe - Pe_vortex)^beta
ax = axes[1]

Pe_above = np.linspace(Pe_vortex, 25, 300)
for beta, ls, label in [(0.5, '-', 'β=0.5 (mean-field)'),
                         (1.0, '--', 'β=1.0 (linear)'),
                         (1.5, ':', 'β=1.5')]:
    omega = (Pe_above - Pe_vortex) ** beta
    omega /= omega.max()  # normalize
    ax.plot(Pe_above, omega, 'k', ls=ls, lw=2, label=label)

# Substrate markers
for name, pe_s, alpha_s, color, marker in substrates:
    if pe_s <= 0:
        continue
    if pe_s < Pe_vortex:
        ax.axvline(x=pe_s, color=color, lw=1.2, ls=':', alpha=0.7)
        ax.text(pe_s, 0.05, name, fontsize=8, color=color, rotation=90,
                va='bottom', ha='right')
    else:
        # Vortex-capable substrates: mark omega value at beta=0.5
        omega_s = ((pe_s - Pe_vortex) / (25 - Pe_vortex)) ** 0.5
        ax.scatter(pe_s, omega_s, s=140, marker=marker, color=color,
                   edgecolors='white', linewidth=1.0, zorder=8)
        ax.text(pe_s + 0.3, omega_s, name, fontsize=8.5, color=color, va='center')

ax.axvline(x=Pe_vortex, color='#6c3483', lw=2, ls='--', label='Pe_vortex = 4')
ax.set_xlabel('Pe', fontsize=12)
ax.set_ylabel('Normalized vortex amplitude |Ω| (a.u.)', fontsize=11)
ax.set_title('Continuous vortex onset: |Ω| ∝ (Pe − 4)^β (DEM-21)\n'
             'ETH (3.74) just below threshold — no stable creator economy', fontsize=10)
ax.set_xlim(0, 25)
ax.set_ylim(-0.05, 1.1)
ax.legend(fontsize=9, loc='upper left')
ax.grid(True, ls=':', alpha=0.3)

# Shade pre-vortex region
ax.fill_between([0, Pe_vortex], -0.05, 1.1, alpha=0.06, color='gray')
ax.text(Pe_vortex/2, 0.55, 'Sub-vortex\n(no viral cycles)', ha='center',
        fontsize=9, color='gray')

plt.suptitle(
    'Vortex threshold analysis — Pe_vortex = 4 divides viral-capable from non-viral substrates\n'
    'ETH at 93.5% of threshold explains no stable DeFi creator economy (DEM-21)',
    fontsize=10, y=1.01,
)
plt.tight_layout()
plt.savefig('nb18_vortex_proximity.svg', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: nb18_vortex_proximity.svg')

## Repulsive Void: Structural Impossibility of Demon Phases

For Pe < 0 (nb16: JW at Pe=−8.92, Mainline at Pe=−3.23):

- **Filling fraction**: f_D = ρ_D / (2·Pe)^(3/2) is imaginary (negative under radical).  
  The filling fraction concept is undefined — there is no "available phase space" to fill.

- **Coupling parameter**: Γ_D = 2α · exp(−Pe / ρ_D^(2/3)). With Pe < 0,  
  the exponent is +|Pe|/ρ_D^(2/3) > 0, so exp(+) grows as ρ_D → 0.  
  Γ_D → ∞ as ρ_D → 0 (divergent coupling at low density — repulsion, not cohesion).

- **Crystal onset**: ρ_D^(crystal) = (Pe/ln(2α/Γ_c))^(3/2). With Pe < 0 and ln(2α/Γ_c) > 0:  
  (Pe/ln(q))^(3/2) is imaginary. No crystal solution exists.

**Physical interpretation (nb16 connection):**  
The same exit gradient (Pe < 0) that drives observer retention loss **also** prevents demon crystallization.  
JW has no creator economy, no influencer layer, no viral cycles — not by policy alone, but structurally.  
The void repels both observers and demons.

This connects to DEM-21: |Ω| ∝ (Pe − 4)^β is undefined for Pe < 0.  
Repulsive voids have |Ω| = 0 — no vortex. No viral cycle possible at any density.

In [ ]:
# Numerically verify the Pe < 0 divergence
print('Pe < 0 behavior: Gamma_D at small rho_D')
print('Expected: diverges as rho_D -> 0 (repulsion, not cohesion)\n')

Pe_jw = -8.92
alpha_jw = 0.5  # low engagement by design (JW discourages external engagement)
rho_test = [0.001, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0]

print(f'{"rho_D":>10} {"Gamma_D (Pe=-8.92, alpha=0.5)":>35} {"f_D":>20}')
print('-' * 68)
for rho in rho_test:
    gD = coupling_param(rho, Pe_jw, alpha_jw)
    # f_D would be imaginary — compute magnitude for display
    try:
        fD_val = filling_fraction(rho, abs(Pe_jw))  # |Pe| for display only
        fD_str = f'{fD_val:.6f} (using |Pe|, not physical)'
    except:
        fD_str = 'undefined'
    print(f'{rho:>10.3f} {gD:>35.4f} {fD_str:>20}')

print()
print('As rho_D -> 0: Gamma_D -> inf (repulsive divergence, not cohesive approach)')
print('As rho_D -> inf: Gamma_D -> 2*alpha (same as Pe>0 case — high density converges)')
print()
print('No demon crystal can form in repulsive voids.')
print('No vortex can form (Pe=-8.92 << Pe_vortex=4).')
print('JW creator economy is structurally impossible by Pe alone, not just by policy.')

# Compare to a normal attractive void at same rho_D range
print()
print('Comparison: ETH (Pe=3.74, alpha=2.0)')
print(f'{"rho_D":>10} {"Gamma_D (ETH)":>20} {"f_D":>15} {"phase":>15}')
print('-' * 62)
Pe_eth = 3.74
alpha_eth = 2.0
Gamma_c_ref = 2.0
for rho in rho_test:
    gD = coupling_param(rho, Pe_eth, alpha_eth)
    fD = filling_fraction(rho, Pe_eth)
    if fD < 1.0:
        phase = 'Gas'
    elif gD >= Gamma_c_ref:
        phase = 'Crystal'
    elif Pe_eth >= Pe_vortex:
        phase = 'Vortex fluid'
    else:
        phase = 'Fluid'
    print(f'{rho:>10.3f} {gD:>20.6f} {fD:>15.6f} {phase:>15}')

## Summary

### Demon lattice phase classification — key results

**1. Four phases from Paper 9 §6.8.2, all implemented from exact formulas.**

| Phase | Condition | Platform Example |
|-------|-----------|------------------|
| Gas | f_D < 1 | Any platform at low demon density |
| Fluid | f_D ≥ 1, Γ_D < Γ_c | Moderate engagement platforms |
| Crystal | Γ_D ≥ Γ_c | Gaming (Pe=4.4), Social Media (Pe=6), Gambling (Pe=12) |
| Crystal+Vortex | Γ_D ≥ Γ_c, Pe > 4 | YouTube, TikTok, Instagram creator economies |
| Repulsive (not a phase) | Pe < 0 | JW (Pe=−8.92), Mainline (Pe=−3.23) — no demon ecosystem |

**2. Pe-independent results (new analytic findings):**

- **At gas/fluid transition**: Γ_D = 2α/√e for every substrate at every Pe.  
  Whether crystal is possible depends only on α vs Γ_c, not on Pe.

- **Crystal/gas onset ratio**: ρ_D^(crystal)/ρ_D^(gas) = 1/(2·ln(q))^(3/2) is Pe-independent.  
  Platforms with the same α/Γ_c ratio need the same fraction of fluid space to crystallize,  
  regardless of their Pe.

**3. ETH sits at 93.5% of Pe_vortex = 4.**  
Sub-vortex: no self-sustaining creator economy. Explains DeFi's lack of stable influencer layer.  
Gaming at Pe=4.4 is the first substrate above the vortex threshold — explains Twitch/YouTube.

**4. Repulsive voids (Pe < 0) cannot sustain demon phases.**  
f_D undefined. Crystal onset imaginary. No vortex condition reachable.  
JW's absence of creator economy is **structural** (by Pe < 0), not just policy.  
Same exit gradient (nb16) causes both retention loss and demon-phase impossibility.

**5. Continuous vortex onset (DEM-21):**  
|Ω| ∝ (Pe − 4)^β. Testable with platform-level creator revenue concentration vs Pe.  
Continuous onset preferred over step-function (ΔAIC > 2 criterion from Paper 9).

### For 'The Congregation Effect' paper

The demon lattice analysis adds to nb16 (repulsive void):
- JW cannot crystallize demons: **no stable ecosystem of paid religious content creators**.  
  This is a falsifiable, numerical prediction (DEM-20, DEM-21 applied to religious sector).
- Buddhist at Pe=0 sits on the null-void boundary: no crystal phase at any density  
  (gas/fluid transition happens but Γ_D stays at 2α/√e — below Γ_c for typical α).
- High-retention denominations (Hindu Pe=30, Jewish Pe≈20) support crystal+vortex:  
  predicts vibrant creator ecosystems (religious media, YouTube channels, podcasts).

### Pending: nb19 (demon plasma frequency)

Next step: linearize the Lotka-Volterra system around equilibrium demon density ρ_D*  
to compute ω_D (plasma frequency) per substrate. Crystal phase is the scaffold;  
ω_D gives the timescale of oscillations within it.